In [1]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:

In [2]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
pip install datasets


In [4]:
from datasets import Dataset, DatasetDict
# Load and split the dataset
from datasets import load_dataset
import json

# Opening JSON file
f = open('cnn.json')

# returns JSON object as
# a dictionary
data = json.load(f)

# Create a dictionary-like structure with 'question' and 'answer' columns
formatted_data = {
    "question": [item["question"] for item in data],
    "answer": [item["answer"] for item in data]
}

# Create a dataset using the formatted data
dataset = Dataset.from_dict(formatted_data)

# Create a DatasetDict with a 'train' split
dataset = DatasetDict({'train': dataset})

# Print information about the dataset
print(dataset)



DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 20
    })
})


In [5]:
# Load and split the dataset
dataset = dataset["train"].train_test_split(test_size=0.2)



In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 16
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 4
    })
})


In [7]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
# We prefix our tasks with "answer the question"
prefix = "answer the question: "

# Define our preprocessing function
def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Map the preprocessing function across our dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [9]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [10]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.334200,0.299548,0.100390,0.211575,0.245291
2,No log,3.265941,0.353716,0.088003,0.244569,0.283289


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=8, training_loss=3.095940351486206, metrics={'train_runtime': 11.6668, 'train_samples_per_second': 2.743, 'train_steps_per_second': 0.686, 'total_flos': 238172209152.0, 'train_loss': 3.095940351486206, 'epoch': 2.0})

In [11]:
trainer.evaluate()

{'eval_loss': 3.2659406661987305,
 'eval_rouge1': 0.35371643829550986,
 'eval_rouge2': 0.08800346144440055,
 'eval_rougeL': 0.2445687952600395,
 'eval_rougeLsum': 0.2832892258541354,
 'eval_runtime': 1.385,
 'eval_samples_per_second': 2.888,
 'eval_steps_per_second': 0.722,
 'epoch': 2.0}

In [15]:
model.save_pretrained("./results")


In [44]:
tokenizer.save_pretrained("./tokens")


('./tokens/tokenizer_config.json',
 './tokens/special_tokens_map.json',
 './tokens/spiece.model',
 './tokens/added_tokens.json')

In [47]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the trained model and tokenizer
model_path = "./results"  # Replace with the path to your trained model
tokenizer_path = "/content/tokens"  # Replace with the path to your trained tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

# Input question for testing
input_question = "What is bert?"

# Prefix for your task
prefix = "answer the question: "

# Tokenize and preprocess the input
input_text = prefix + input_question
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True)

# Generate a response from the model
with torch.no_grad():
    # Ensure you are using input_ids as the input, not 'inputs'
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50)

# Decode the generated response
generated_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated response
print("Generated Response:", generated_response)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated Response: Bert is a genus of fungi in the family Geometridae.


In [ ]:
# Save our tokenizer and create model card
tokenizer.save_pretrained()
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()



SyntaxError: ignored

In [ ]:
from transformers import pipeline
from random import randrange

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="#model", device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-large summary:\n{res[0]['summary_text']}")


OSError: ignored